In [1]:
import sys
sys.path.insert(0, '/tf/utils/')

In [2]:
base_shape_size = 8192
ws = 255
ol = 128
input_shape = (128, 64, 1)

In [3]:
from utils import calculate_stft_magnitude_and_phase, reconstruct_signal_from_stft
from sound import Sound
from data_generators import NoisyTargetGenerator
from artigos.testes import create_model
import tensorflow as tf
# tf.debugging.set_log_device_placement(True)

from tensorflow.keras.optimizers import Adam

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Audio
from IPython import display
import librosa

In [4]:
sound_base_train = Sound('/tf/Dados/Vozes/train/', '/tf/Dados/Ruido/train/', base_shape_size)
sound_base_val = Sound('/tf/Dados/Vozes/val/', '/tf/Dados/Ruido/val/', base_shape_size)

Loading Noise Files: 100%|██████████| 5695/5695 [00:04<00:00, 1283.17it/s]
/tf/utils/sound.py:62: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  clean_sounds = [sound for sound in clean_sounds if sound != self.TOO_SHORT_ERROR]
/tf/utils/sound.py:74: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  noise_sounds = [sound for sound in noise_sounds if sound != self.TOO_SHORT_ERROR]
Loading Noise Files: 100%|██████████| 1627/1627 [00:02<00:00, 739.23it/s] 


In [5]:
data_generator_train = NoisyTargetGenerator(sound_base_train.clean_sounds, sound_base_train.noise_sounds)
data_generator_val = NoisyTargetGenerator(sound_base_val.clean_sounds, sound_base_val.noise_sounds)

In [6]:
def generate_images(model, test_input, tar):
    
    prediction = model(test_input.reshape(-1, 128, 64, 1), training=True)
    plt.figure(figsize=(22, 7))
    
    display_list = [test_input[0], tar[0], prediction[0]]
    title = ['Log Power Spectrum - Som ruidoso', 'Log Power Spectrum - Som original', 'Log Power Spectrum - Som filtrado']
    
    for i in range(3):
        plt.subplot(1, 3, i+1)
        plt.title(title[i])
        # Getting the pixel values in the [0, 1] range to plot.
        plt.imshow(10 * np.log10((display_list[i][..., 0])**2), aspect='auto', cmap='inferno')
        plt.colorbar(format='%+2.0f dB')  # Removi a variável 'im' e 'axs[0]'
        plt.axis('off')
    
    plt.show()

## Estrutura do modelo para 3 canais

![Descrição da imagem](model_structure.png)

In [7]:
# with tf.device('/GPU:0'):
model = create_model(input_shape)

Input = (None, 128, 64, 1)
Conv block = (None, 128, 64, 64)
Channel Attention = (None, 128, 64, 64)
Channel Attention Last CNN = (None, 128, 64, 1)
First phase = (None, 128, 64, 2)



ValueError: Exception encountered when calling layer "multi_scale_feature_extraction" (type Multi_scale_feature_extraction).

in user code:

    File "/tf/utils/artigos/testes.py", line 225, in call  *
        up_sample_16 = self.msfe_16(X)
    File "/usr/local/lib/python3.8/dist-packages/keras/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filertr2potj.py", line 11, in tf__call
        unet = ag__.converted_call(ag__.ld(self).unet_layers_lstm, (ag__.ld(avg_pool),), None, fscope)

    ValueError: Exception encountered when calling layer 'avg_pool__unet__upsample_msfe' (type Avg_pool_Unet_Upsample_msfe).
    
    in user code:
    
        File "/tf/utils/artigos/testes.py", line 207, in call  *
            unet = self.unet_layers_lstm(avg_pool)
        File "/usr/local/lib/python3.8/dist-packages/keras/utils/traceback_utils.py", line 70, in error_handler  **
            raise e.with_tra

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, dpi=64)

In [ ]:
class GenerateImagesCallback(tf.keras.callbacks.Callback):
    def __init__(self, x_val, y_val):
        self.x_val = x_val
        self.y_val = y_val

    def on_epoch_end(self, epoch, logs=None):
        indice_aleatorio = np.random.choice(self.x_val.shape[0])
        amostra_noisy_module = self.x_val[indice_aleatorio][np.newaxis, ...]
        amostra_original_module = self.y_val[indice_aleatorio][np.newaxis, ...]
        generate_images(self.model, amostra_noisy_module, amostra_original_module)

In [ ]:
# Gera um novo lote de validação para cada época
validation_batch = next(data_generator_val.generate_sample_completo(batch_size=32))
x_val, y_val = validation_batch

In [ ]:
steps_per_epoch_train = len(sound_base_train.clean_sounds)
steps_per_epoch_validation = len(sound_base_val.clean_sounds)

In [ ]:
best_models_path = "/tf/Etapa 3/Artigos/Testes/model_checkpoints/"
callbacks_lst = [
                 tf.keras.callbacks.ModelCheckpoint(filepath=best_models_path+"best_PRIDNet_blindnoise_128x64.h5", save_freq=100, save_weights_only=False),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', min_lr=0.0000009, min_delta=0.0001, factor=0.70, patience=3, verbose=1, mode='min'),
    # tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, min_delta=0.0001, patience=10),
    GenerateImagesCallback(x_val, y_val)
]

In [ ]:
def weighted_msle(y_true, y_pred):
    # Define o peso para penalizar mais as subestimações
    w = 2.0

    # Evita valores negativos adicionando 1 antes de aplicar o log
    log_true = 20 * tf.math.log(y_true + 1)
    log_pred = 20 * tf.math.log(y_pred + 1)

    # Cálculo do erro
    error = log_true - log_pred

    # Aplica pesos diferentes para superestimação e subestimação
    weighted_error = tf.where(error > 0, w * error, error)

    # Retorna a média do erro logarítmico quadrado ponderado
    return tf.reduce_mean(tf.abs(weighted_error))

In [ ]:
# model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=Adam(learning_rate=0.0001))
# model.compile(loss=tf.keras.losses.MeanAbsoluteError(), optimizer=Adam(learning_rate=0.0009))
model.compile(
    loss='mse',
    optimizer=Adam(
        learning_rate=1e-05,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-08,
    )
)
# model.compile(loss=tf.keras.losses.MeanSquaredLogarithmicError(), optimizer=Adam())

In [ ]:
batch_size = 8
steps_per_epoch = len(sound_base_train.clean_sounds) // batch_size
epochs = 200

val_data = next(data_generator_val.generate_sample_completo(batch_size=4 * batch_size, include_clean=False, only_return_mudule=True))

model.fit(data_generator_train.generate_sample_completo(batch_size=batch_size, include_clean=False, only_return_mudule=True),
          steps_per_epoch=steps_per_epoch,
          epochs=epochs,
          validation_data=val_data,
          callbacks=callbacks_lst
         )

In [ ]:
validation_batch = next(data_generator_val.generate_sample_completo(batch_size=8))
x_test, y_test = validation_batch

x_test = x_test[0, ...]
y_test = y_test[0, ...]

x_test = x_test[np.newaxis, ...]
y_test = y_test[np.newaxis, ...]

In [ ]:
print(np.max(x_test[:, :, : , 0]))
print(np.min(x_test[:, :, : , 0]))
print(np.max(y_test[:, :, : , 0]))
print(np.min(y_test[:, :, : , 0]))

In [ ]:
x_test.shape

In [ ]:
F_f = model.predict(x_test[:, :, :, 0].reshape(-1, 128, 64, 1))

In [ ]:
print(np.max(F_f))
print(np.min(F_f))
print(F_f.shape)

In [ ]:
A = y_test.reshape(y_test.shape[1], y_test.shape[2], y_test.shape[3])[:, :, 0]
phi = (y_test.reshape(y_test.shape[1], y_test.shape[2], y_test.shape[3])[:, :, 1] - 0.5) * 2 * np.pi

A_f = F_f.reshape(F_f.shape[1], F_f.shape[2], F_f.shape[3])[:, :, 0]
# phi_f = (F_f.reshape(F_f.shape[1], F_f.shape[2], F_f.shape[3])[:, :, 1] - 0.5) * 2 * np.pi

A_n = x_test.reshape(x_test.shape[1], x_test.shape[2], x_test.shape[3])[:, :, 0]
phi_n = (x_test.reshape(x_test.shape[1], x_test.shape[2], x_test.shape[3])[:, :, 1] - 0.5) * 2 * np.pi

In [ ]:
print(np.max(A_f))
# print(np.max(phi_f))
print(np.min(A_f))
# print(np.min(phi_f))

In [ ]:
generate_images(model, x_test, y_test)

In [ ]:
s = reconstruct_signal_from_stft(A, phi)
s_n = reconstruct_signal_from_stft(A_n, phi_n)
s_f = reconstruct_signal_from_stft(A_f, phi_n)

In [ ]:
# Criando figuras e eixos separados para cada array
fig, (ax1, ax2, ax3) = plt.subplots(3, 1)

# Plotando o primeiro array
ax1.plot(s)
ax1.set_ylabel('Sinal de voz ruidoso')

# Plotando o primeiro array
ax2.plot(s_n)
ax2.set_ylabel('Sinal de voz ruidoso')

ax3.plot(s_f)
ax3.set_ylabel('Sinal de voz filtrado')

# Exibindo os gráficos
plt.show()

In [ ]:
Audio(data=s, rate=8000)

In [ ]:
Audio(data=s_n, rate=8000)

In [ ]:
Audio(data=s_f, rate=8000)